In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("day1").getOrCreate()


In [4]:
par_df = spark.read.parquet('data.parquet')
par_df = spark.read.format('parquet').load('data.parquet')
par_df.show()
par_df.printSchema()

+--------+----------+--------+----------------+
|      id|     tdate|category|         product|
+--------+----------+--------+----------------+
|00000000|06-26-2011|Exercise|Gymnastics Rings|
|00000002|06-01-2011|Exercise|Gymnastics Rings|
+--------+----------+--------+----------------+

root
 |-- id: string (nullable = true)
 |-- tdate: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)



In [10]:
from pyspark.sql.functions import min, max, avg, col, expr

from pyspark.sql.types import IntegerType

orc_df = spark.read.orc('data.orc')
# par_df = spark.read.format('orc').load('data.orc')
# orc_df.show()
# orc_df.printSchema()

df1= orc_df.withColumn('age', col('age').cast(IntegerType()))
# df1.printSchema()

df = orc_df.agg(min("age").alias('min_age'),
                max("age").alias('max_age'),
                avg('age').alias('avg_age'))

df.show()

orc_df.filter(orc_df.city.contains('New')).show()

+-------+-------+-------+
|min_age|max_age|avg_age|
+-------+-------+-------+
|     11|     33| 21.375|
+-------+-------+-------+

+----------+---------+--------------------+--------------+-----------+-------+-----+-----+---+------------+------------+--------------------+--------------------+
|first_name|last_name|        company_name|       address|       city| county|state|  zip|age|      phone1|      phone2|               email|                 web|
+----------+---------+--------------------+--------------+-----------+-------+-----+-----+---+------------+------------+--------------------+--------------------+
|    Arlene|  Klusman|Beck Horizon Buil...|    3 Secor Rd|New Orleans|Orleans|   LA|70112| 20|504-710-5840|504-946-1807|arlene_klusman@gm...|http://www.beckho...|
|  Terrilyn|Rodeigues|      Stuart J Agins|3718 S Main St|New Orleans|Orleans|   LA|70130| 33|504-463-4384|504-635-8518|terrilyn.rodeigue...|http://www.stuart...|
+----------+---------+--------------------+-----------

In [11]:
avro_df = spark.read.format("avro").load("data.avro")

avro_df.show()

AnalysisException: Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of Apache Avro Data Source Guide.

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Read Avro") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0") \
    .getOrCreate()

avro_df = spark.read.format("avro").load("data.avro")
avro_df.show()


AnalysisException: Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of Apache Avro Data Source Guide.

In [14]:
orc_df= orc_df.withColumnRenamed('zip', 'pincode')

orc_df.createOrReplaceTempView("users")

results = spark.sql(" select min(age) min_age, max(age) max_age from users")
results.show()

orc_df.count()
orc_df.printSchema()

orc_df = orc_df.select('first_name', 'phone1', 'company_name')
orc_df.show()


+-------+-------+
|min_age|max_age|
+-------+-------+
|     11|     33|
+-------+-------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- pincode: string (nullable = true)
 |-- age: string (nullable = true)
 |-- phone1: string (nullable = true)
 |-- phone2: string (nullable = true)
 |-- email: string (nullable = true)
 |-- web: string (nullable = true)

+----------+------------+--------------------+
|first_name|      phone1|        company_name|
+----------+------------+--------------------+
|   Solange|504-979-9175|   Mosocco, Ronald A|
|    Arlene|504-710-5840|Beck Horizon Buil...|
|     Larae|985-890-7262|Lehigh Furn Divsn...|
| Willodean|337-253-8384|            Magnuson|
|  Terrilyn|504-463-4384|      Stuart J Agins|
|  Kayleigh|337-740-9323|Den

In [15]:
from pyspark.sql.functions import *
data = ((1, 'charan', 28), (2, 'dheeraj', 17), (3, 'ram' , 61) )
columns = ['id', 'name', 'age']
df = spark.createDataFrame(data, columns)

results = df.withColumn('status', when(df.age <= 17, 'minor') \
                                  .when((df.age >=18) & (df.age <=60), 'major') \
                                  .otherwise('sr citizen') )
results.show()

+---+-------+---+----------+
| id|   name|age|    status|
+---+-------+---+----------+
|  1| charan| 28|     major|
|  2|dheeraj| 17|     minor|
|  3|    ram| 61|sr citizen|
+---+-------+---+----------+

